# Input & Label

In [1]:
from rtmag.paper.load import load_input_label, MyModel

In [2]:
input_files, label_files = load_input_label("/mnt/f/isee_dataset/11158")
len(input_files), len(label_files)

(66, 66)

In [3]:
idx = 13
input_files[idx]

PosixPath('/mnt/f/isee_dataset/11158/input/input_11158_20110214_000000.npz')

# Model prediction & Label (NLFFF, Potential)

In [4]:
from time import time

In [12]:
# mm = MyModel("/home/mgjeon/space/workspace/base/pino_CC_Hnorm_Unit_2/best_model.pt")
mm = MyModel("/home/mgjeon/space/workspace/base_important/uno_pi_cc_hnorm_unit_aug/model_75.pt")
# mm2 = MyModel("/home/mgjeon/space/workspace/base/uno_pi_cc_hnorm_unit_aug/model_60.pt", clip=6000)

Model loaded from epoch 75


In [13]:
start_time = time()
b = mm.get_pred(input_files[idx])
print(f"{time() - start_time} seconds")

0.890526294708252 seconds


In [14]:
start_time = time()
B = mm.get_label(label_files[idx])
print(f"{time() - start_time} seconds")

1.2648975849151611 seconds


In [15]:
start_time = time()
Bp = mm.get_pot(label_files[idx])
print(f"{time() - start_time} seconds")

1.2544984817504883 seconds


In [16]:
start_time = time()
dx, dy, dz, dV = mm.get_dV(input_files[idx])
print(f"{time() - start_time} seconds")

0.0037012100219726562 seconds


# Quantitative Evaluation

In [17]:
import rtmag.paper.metric as m

In [18]:
start_time = time()
res = m.evaluate(b, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.96
E_n'      : 0.69
E_m'      : 0.60
eps       : 0.94
1.1196656227111816 seconds


In [12]:
start_time = time()
res = m.evaluate_energy(b, B, Bp, dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 0.80
ref_E_1e33: 0.85
pot_E_1e33: 0.71
dV_1e23   : 1.53
pred_E_unit: 132100952340.47
ref_E_unit: 139871515001.88
pot_E_unit: 116420454927.01
C_vec     : 0.96
C_cs      : 0.96
E_n_prime : 0.69
E_m_prime : 0.60
eps       : 0.94
rel_l2_err: 0.27
2.3176681995391846 seconds


In [13]:
start_time = time()
res = m.evaluate_energy(b[:, :, :128, :], B[:, :, :128, :], Bp[:, :, :128, :], dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 0.79
ref_E_1e33: 0.84
pot_E_1e33: 0.70
dV_1e23   : 1.53
pred_E_unit: 130748459372.66
ref_E_unit: 139057338988.71
pot_E_unit: 115651620541.42
C_vec     : 0.96
C_cs      : 0.93
E_n_prime : 0.70
E_m_prime : 0.63
eps       : 0.94
rel_l2_err: 0.27
1.1510465145111084 seconds


In [14]:
start_time = time()
res = m.evaluate_energy(b[:, :, :64, :], B[:, :, :64, :], Bp[:, :, :64, :], dV, isprint=True)
print(f"{time() - start_time} seconds")

pred_E_1e33: 0.76
ref_E_1e33: 0.82
pot_E_1e33: 0.68
dV_1e23   : 1.53
pred_E_unit: 126019837596.27
ref_E_unit: 135255310580.55
pot_E_unit: 112190644904.81
C_vec     : 0.96
C_cs      : 0.88
E_n_prime : 0.69
E_m_prime : 0.50
eps       : 0.93
rel_l2_err: 0.27
0.615614652633667 seconds


In [15]:
(dz/1e8) * 64

34.3470344543457

In [16]:
start_time = time()
res = m.evaluate(b[:, :, :64, :], B[:, :, :64, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.88
E_n'      : 0.69
E_m'      : 0.50
eps       : 0.93
0.2608950138092041 seconds


In [17]:
start_time = time()
res = m.evaluate(b[128:-128, 64:-64, :128, :], B[128:-128, 64:-64, :128, :])
print(f"{time() - start_time} seconds")

C_vec     : 0.97
C_cs      : 0.96
E_n'      : 0.75
E_m'      : 0.69
eps       : 0.92
0.14307069778442383 seconds


In [18]:
m.metrics_j(b)

(0.8032389391042999, 72.26363457930378, 34.400898956836244)

In [19]:
m.metrics_j(B)

(0.685718401941427, 72.86644558109298, 12.196899046200134)

In [20]:
m.metrics_j(Bp)

(0.8039919919220125, 1.5027680251842161, 1.5461015324053275)

In [21]:
start_time = time()
bf = mm.fine_tuning(dx=dx/1e8, dy=dy/1e8, dz=dz/1e8, 
                    w_reg=0.01, w_bc=10, w_ff=1, w_div=1,
                    lr=1e-5, n_epoch=10)
print(f"{time() - start_time} seconds")

0
1
2
3
4
5
6
7
8
9
8.446494102478027 seconds


In [22]:
m.metrics_j(bf)

(0.8082438380538803, 66.89454959689958, 35.555600650722525)

In [23]:
start_time = time()
res = m.evaluate(bf, B)
print(f"{time() - start_time} seconds")

C_vec     : 0.96
C_cs      : 0.94
E_n'      : 0.68
E_m'      : 0.51
eps       : 0.94
1.0748038291931152 seconds
